In [13]:
!pip install scikit-learn gensim matplotlib seaborn pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 17.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0;

In [2]:

import pandas as pd
df = pd.read_csv('/content/bbc_news_data.csv')  # Use 200 articles for speed
print(df.head())

   Unnamed: 0                                               Text  \
0           0  US consumer confidence up\n\nConsumers' confid...   
1           1  The 'ticking budget' facing the US\n\nThe budg...   
2           2  Mitsubishi in Peugeot link talks\n\nTrouble-hi...   
3           3  BMW reveals new models pipeline\n\nBMW is prep...   
4           4  World leaders gather to face uncertainty\n\nMo...   

                                             Summary  
0  Wal-Mart, the largest US retailer, has said it...  
1  Brute force budget cuts or spending caps would...  
2  Trouble-hit Mitsubishi Motors is in talks with...  
3  Typically it takes about three years from when...  
4  More than 2,000 business and political leaders...  


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
texts = df['Text']
vectorizer = CountVectorizer(max_features=1000, stop_words='english')
bow_matrix = vectorizer.fit_transform(texts)
print(bow_matrix.shape)  # (200, 1000)
print(vectorizer.get_feature_names_out()[:10])  # First 10 words

(2225, 1000)
['000' '10' '100' '11' '12' '13' '14' '15' '16' '17']


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
tfidf_matrix = vectorizer.fit_transform(texts)
print(tfidf_matrix.shape)
print(vectorizer.get_feature_names_out()[:10])

(2225, 1000)
['000' '10' '100' '11' '12' '13' '14' '15' '16' '17']


In [6]:
import re
from gensim.models import Word2Vec
# Preprocess (from Chapter 2 skills)
texts = df['Text'].apply(lambda x: re.sub(r'http\S+|[^\x00-\x7F]+|[.,!?]', '', x.lower()))
tokenized_texts = [text.split() for text in texts]
model = Word2Vec(tokenized_texts, vector_size=100, window=5, min_count=5)
print(model.wv['market'])  # Vector for "market"
print(model.wv.most_similar('market', topn=5))  # Similar words

[-0.7580614  -0.46328846 -0.16201992  0.30051965 -0.27837047  0.31323785
  0.36420265  0.68751067 -0.7091084  -0.66670966 -0.06539743 -0.2151784
 -0.04964463 -0.39891583 -0.21194486 -0.10011972 -0.49945042  0.8452056
  0.01825659 -0.37681946 -0.14904799 -0.26714993  1.0820867  -1.1476531
  0.6415985  -0.44779834  0.34342757  0.6049159  -0.6721146  -0.00857114
  0.2910533   0.21107951  0.7197886  -0.6757999  -0.16668555  0.14617321
 -1.0004002  -0.2459907  -0.4880838  -0.02149073  0.38531977 -0.6840683
 -0.52103513 -0.25027612  0.4416578  -0.4242215   0.1656143  -0.4507223
 -0.18233964  0.09088071  0.13498309 -0.02028232 -0.14351861 -0.3086081
 -0.18209137 -0.1758936   0.21805595 -0.02816815 -1.1756265   0.02277546
  0.30353752 -0.32909873  0.90321064 -0.6483222  -0.3216958   0.38963005
  0.4155281   0.30359912 -0.8283791   0.43221182  0.02336066  0.5658029
  0.11600278  0.07292151  0.08520783  0.18338297  0.7872399   0.20134814
  0.05976127  0.44556963  0.020968   -0.3087853  -0.125351

In [13]:
# from sklearn.manifold import TSNE
# import matplotlib.pyplot as plt
# words = list(model.wv.index_to_key)[:50]
# embeddings = [model.wv[word] for word in words]
# tsne = TSNE(n_components=2, random_state=42)
# embeddings_2d = tsne.fit_transform(embeddings)
# plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1])
# for i, word in enumerate(words):
#     plt.annotate(word, (embeddings_2d[i, 0], embeddings_2d[i, 1]))
# plt.show()

In [11]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Load data
df = pd.read_csv('/content/bbc_news_data.csv')[:200]
df['Text'] = df['Text'].apply(lambda x: re.sub(r'http\S+|[^\x00-\x7F]+|[.,!?]', '', x.lower()))

# TF-IDF
vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df['Text'])
feature_names = vectorizer.get_feature_names_out()
top_terms = []
for Summary in df['Summary'].unique():
    Summary_texts = df[df['Summary'] == Summary]['Text']
    Summary_tfidf = vectorizer.transform(Summary_texts).toarray()
    avg_tfidf = Summary_tfidf.mean(axis=0)
    top_indices = avg_tfidf.argsort()[-10:]
    terms = [feature_names[i] for i in top_indices]
    top_terms.append([Summary, terms])
pd.DataFrame(top_terms, columns=['Summary', 'top_terms']).to_csv('news_analysis.csv')

# Word2Vec
tokenized_texts = [text.split() for text in df['Text']]
model = Word2Vec(tokenized_texts, vector_size=100, window=5, min_count=5)
words = list(model.wv.index_to_key)[:50]
embeddings = [model.wv[word] for word in words]

# t-SNE
# tsne = TSNE(n_components=2, random_state=42)
# embeddings_2d = tsne.fit_transform(embeddings)
# plt.figure(figsize=(10, 8))
# plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1])
# for i, word in enumerate(words):
#     plt.annotate(word, (embeddings_2d[i, 0], embeddings_2d[i, 1]))
# plt.savefig('tsne_plot.png')
plt.show()